In [1]:
import pandas as pd
import numpy as np
import glob
from collections import defaultdict

In [2]:
#将年月日连接起来转化为 '%Y-%m-%d'格式
def concat_date(dvsdf,concatname):
    YMD=[]
    for i in range(dvsdf.shape[0]):
        ymd=str(dvsdf['年份'][i])+'-'+str(dvsdf[concatname][i]).zfill(4)[:2]+'-'+str(dvsdf[concatname][i])[-2:]
        YMD.append(ymd)
    return YMD

import datetime
def date_period(time1,time2):
    a1 = time1
    a2 = time2
    b1 = a1.split('-')
    b2 = a2.split('-')
    c1=datetime.date(int(b1[0]), int(b1[1]), int(b1[2]))   #开始的日期de
    c2=datetime.date(int(b2[0]), int(b2[1]), int(b2[2]))  #  结束的日期
    result=(c2-c1).days
    return result

In [3]:
#计算日序DOY
import datetime
def DOY_cal(time):
    #获取年、月、日.      计算这是这一年的第几天.       算法：前几个月的总天数 + 当月天数
    a=datetime.datetime.strptime(time,'%Y-%m-%d').strftime('%Y%m%d')
    year=int(a[:4])
    month=int(a[4:6])
    day=int(a[6:])

    day_of_second = 29 if year % 4 == 0 and year % 100 != 0 or year % 400 == 0 else 28
    days_of_month = (31, day_of_second, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31)

    total_days = sum(days_of_month[: month - 1])
    total_days += day
    return (str(total_days))

In [4]:
import math
def daylength(lat,DOY):
    rad=0.017453292
    dec=-math.asin(math.sin(23.45*rad)*math.cos(2*math.pi*(DOY+10)/365))
    sinld=math.sin(rad*lat)*math.sin(dec)
    cosld=math.cos(rad*lat)*math.cos(dec)
    aob=sinld/cosld
    if (aob < -1 ):
        D=0
    elif (aob > 1):
        D=24
    else:
        D=12*(1+2*math.asin(aob)/math.pi) 
    D=D+0.8    #d应为天文日长，考虑到暮曙光作用，因此加上日出前后各6度，折算为小时共0.8h
    return D

In [5]:
def Fpho(D):#dayl就是D光周期
    Dc=20
    Do=12.5
    f_pho=(D-Dc)/(Do-Dc)
    return f_pho

In [6]:
#计算逐日热时DTT
def calculate_dtt(dvsdf,Tb,Tc):#这个df表示的是对应的某一天的均温温度
    if dvsdf['TemAver'].values <= Tb:
        dtt=0
    elif dvsdf['TemAver'].values > Tc:
        dtt=Tc-Tb
    else:
        dtt=dvsdf['TemAver'].values-Tb
    return float(dtt)

In [7]:
dvsfilepath=sorted(glob.glob("*csv"))
metefilepath=sorted(glob.glob("*txt"))

In [ ]:
f4=defaultdict(list)
for d,m in zip(dvsfilepath,metefilepath):#表示对‘站号’文件进行遍历
    print("正在进行"+d[*:*]) 
    dvsdf=pd.read_table(d,encoding='gbk',sep=",") #读入dvs数据和气象数据
    metedf1=pd.read_table(m,encoding='gbk',sep='\t')
    metedf1.index=pd.DatetimeIndex(metedf1['ymd'])  
    
    DOYmin=dvsdf["DOY1"].min()+1
    DOYmax=dvsdf["DOY2"].max()
    lat=dvsdf["lat"][0]
    f_min=float(format(Fpho(daylength(lat,DOYmin)),".2f"))
    f_max=float(format(Fpho(daylength(lat,DOYmax)),".2f"))
    if f_min>f_max:
        real=f_min
        f_min=f_max
        f_max=real
    f2=defaultdict(list)

    for Tb in range(2,15,2):
        for Tc in range(30,45,2):
            for f in np.arange(0,1+0.01,0.1):
                for JW in range(400,1500,50):                    
                    f1=defaultdict(list)
#                     print("参数",Tb,Tc,f_pho,JW)
                    for i in range(dvsdf.shape[0]):
                        n=(pd.Timestamp(dvsdf["ymd1"][i])+pd.Timedelta("1day")).date().strftime('%Y-%m-%d')  #原始数据   字符串切片只是为了去掉时间格式中的时分秒
                        m=(pd.Timestamp(dvsdf['ymd2'][i])+pd.Timedelta("30day")).date().strftime('%Y-%m-%d')  #误差不超过30天 
                        period1=date_period(n,m)

                        period_d1=0
                        day1=pd.Timestamp(dvsdf["ymd1"][i])
                        f_pho=f
                        jiwen=JW
                        for k1 in range(period1+1):#原本是从0行开始，但是算法俩面左开右闭，n那里+1滑动
                            day1 =day1+pd.Timedelta("1day")
                            daystr1=day1.date().strftime("%Y-%m-%d")
                            metedf4=metedf1[metedf1.index==daystr1]
                            dailydtt=calculate_dtt(metedf4,Tb,Tc)*f_pho
                            jiwen=jiwen-dailydtt
                            period_d1+=1  
                         
                            if jiwen < 0:
                                SimErr1=period_d1-dvsdf["发育天数1"][i]
#                                 print(SimErr1)
                                f1["误差1"].append(SimErr1)
                                break
                        if jiwen > 0:
                            f1["误差1"].append(np.nan) 
                    F1=pd.DataFrame(f1)
                    F1=F1.dropna(axis=0)

                    if F1.shape[0]==0: 
                        break
                    else:
                        s=0
                        ny=len(F1)
                        for y in F1["误差1"]:
                            s+=y**2
                        RMSE=(s/F1.shape[0])**0.5
                        f2["站号"].append(d[-9:-4])
                        f2["参数-Tb"].append(Tb)
                        f2["参数-Tc"].append(Tc)
                        f2["参数-f_pho"].append(f)
                        f2["积温"].append(JW)
                        f2["RMSE"].append(RMSE)
                        f2["预测年数"].append(ny)
#                         print(Tb,Tc,f,JW,RMSE,ny)                   
    F2=pd.DataFrame(f2)
    name1="WOFOST数据/参数1/"+d[-9:-4]+'站点参数表.csv'
    F2.to_csv(name1,index=False,encoding='gbk')
    print(d[*:*],"执行结束  √","     完成时间",datetime.datetime.now())   
print("程序运行结束！")

In [ ]:
parafile=sorted((glob.glob("*csv")))
f4=defaultdict(list)
for p in parafile:
    paradf=pd.read_csv(p,encoding="gbk",sep=",")
    co=paradf[paradf["预测年数"]==paradf["预测年数"].max()]
    a=co[co["RMSE"]==co["RMSE"].min()]
    b=a[a["参数-Tc"]==a["参数-Tc"].max()]
    c=a[a["参数-Tc"]==a["参数-Tc"].max()][a["参数-Tb"]==a["参数-Tb"].min()]
    e=c[c["参数-f_pho"]==c["参数-f_pho"].min()]
    if len(b)>0 and len(c)>0:
        f4["站号"].append(int(e["站号"]))
        f4["最优-Tb"].append(int(e["参数-Tb"]))
        f4["最优-Tc"].append(int(e["参数-Tc"]))
        f4["最优-f_pho"].append(float(e["参数-f_pho"]))
        f4["最优-积温"].append(float(e["积温"]))
        f4["RMSE"].append(float(e["RMSE"]))
    elif len(a)>1 and len(b)>1:
        f4["站号"].append(int(c["站号"]))
        f4["最优-Tb"].append(int(c["参数-Tb"]))
        f4["最优-Tc"].append(int(c["参数-Tc"]))
        f4["最优-f_pho"].append(float(c["参数-f_pho"]))
        f4["最优-积温"].append(float(c["积温"]))    
        f4["RMSE"].append(float(C["RMSE"]))
    elif len(a)>1 and (len(b)==1) :
        f4["站号"].append(int(b["站号"]))
        f4["最优-Tb"].append(int(b["参数-Tb"]))
        f4["最优-Tc"].append(int(b["参数-Tc"]))
        f4["最优-f_pho"].append(float(b["参数-f_pho"]))
        f4["最优-积温"].append(float(b["积温"]))
        f4["RMSE"].append(float(b["RMSE"]))
    else:
        f4["站号"].append(int(a["站号"]))
        f4["最优-Tb"].append(int(a["参数-Tb"]))
        f4["最优-Tc"].append(int(a["参数-Tc"]))
        f4["最优-f_pho"].append(float(a["参数-f_pho"]))
        f4["最优-积温"].append(float(a["积温"]))
        f4["RMSE"].append(float(a["RMSE"]))
    print(p[-14:-9],"执行结束  √","     完成时间",datetime.datetime.now())
F4=pd.DataFrame(f4)
print(F4)
name="WOFOST数据/参数/"+'站点参数VGP阶段.csv'
F4.to_csv(name,index=False,encoding='gbk')

--------------------------------------------------进行WOFOST     VGP模拟  --------------------------------------------

In [18]:
def calculate_dtt_(dvsdf,Tb,Tc):#这个df表示的是对应的某一天的均温温度
    if float(dvsdf['TemAver']) <= float(Tb):
        dtt=0
    elif float(dvsdf['TemAver']) > float(Tc):
        dtt=Tc-Tb
    else:
        dtt=float(dvsdf['TemAver'])-float(Tb)
    return float(dtt)

In [ ]:
for d,m in zip(dvsfilepath,metefilepath):#表示对‘站号’文件进行遍历
    print("正在进行"+d[*:*]) 
    dvsdf=pd.read_table(d,encoding='gbk',sep=",") #读入dvs数据和气象数据
    metedf1=pd.read_table(m,encoding='gbk',sep='\t')
    metedf1.index=pd.DatetimeIndex(metedf1['ymd'])  
    
    Tpara=pd.read_csv("WOFOST数据/参数/站点参数VGP阶段.csv",encoding="gbk")
    x=Tpara[Tpara["站号"]==int(d[-9:-4])]
    Tb=int(x["最优-Tb"])
#     Tb=Tb.values
    Tc=int(x["最优-Tc"])
#     Tc=Tc.values
    f=float(x["最优-f_pho"])
#     f_pho=f_pho.values
    JW=int(x["最优-积温"])
    
#     jiwen=jiwen.values
#     print(Tb,Tc,f_pho,JW)
    f2=defaultdict(list)

    for i in range(dvsdf.shape[0]):
        n=(pd.Timestamp(dvsdf["ymd1"][i])+pd.Timedelta("1day")).date().strftime('%Y-%m-%d')  #原始数据   字符串切片只是为了去掉时间格式中的时分秒
        m=(pd.Timestamp(dvsdf['ymd2'][i])+pd.Timedelta("30day")).date().strftime('%Y-%m-%d')  #误差不超过30天 
        period1=date_period(n,m)

        period_d1=0
        day1=pd.Timestamp(dvsdf["ymd1"][i])
        jiwen=JW
        f_pho=f

#         print(i)
        for k1 in range(period1+1):#原本是从0行开始，但是算法俩面左开右闭，n那里+1滑动
            day1 =day1+pd.Timedelta("1day")
            daystr1=day1.date().strftime("%Y-%m-%d")
            metedf4=metedf1[metedf1.index==daystr1]
            dailydtt=calculate_dtt_(metedf4,Tb,Tc)*f_pho
            jiwen=jiwen-dailydtt
            period_d1+=1  
#             print(period_d1,jiwen)

            if jiwen < 0:
#                 print(i,"----------",jiwen,SimErr1)
                SimErr1=period_d1-dvsdf["发育天数1"][i]
                f2["误差1"].append(SimErr1)
                break
        if jiwen > 0:
            SimErr1=30
            f2["误差1"].append(SimErr1) 
                                      
    F2=pd.DataFrame(f2)
#     print(F2)
    dvsdf=dvsdf.join(F2,how="inner")
    
    name1="WOFOST数据/留一验证/"+d[*:*]+'.csv'
    dvsdf.to_csv(name1,index=False,encoding='gbk')
    print(d[*:*],"执行结束  √","     完成时间",datetime.datetime.now())   
print("程序运行结束！")